In [ ]:
'''
Prepare parameters for model based on deisgn and measurement conditions
Unit: mm
Author: Yitian Shao
Created on 2022.01.17
'''
# %matplotlib notebook 
# %matplotlib notebook 

import time
from os import walk
import os.path as ospa
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal

plt.rcParams.update({'font.size': 14})

from PouchLib import getEpsilon0 

try:
    EPSILON0 = getEpsilon0() # (Farad/m) Free-space permittivity
except:
    EPSILON0 = 0.0000000000088541878128

calib = np.loadtxt('Calibration20210802.txt')
print("Calibration line a = %.16f, b = %.16f" % (calib[0],calib[1]))


In [ ]:
''' Tube infill compensation (optional 2022.08.16) based on area ratio between the tube and pouch '''
TubeLengthValues = np.array([60, 70, 80, 90, 100]) # (mm) length of tube in CAD designs
TubeDesignedWidth = 7.15-1 # (mm) width of tube in CAD designs
PouchBottomWidth = 24.64 # (mm) width of pouch bottom in CAD designs
PouchHeight = 72.71 # (mm) ideal height of the triangle pouch in CAD designs
PouchNeckDist = 51.60 # (mm) distance of pouch neck from the bottom in CAD designs

''' Unzip Volume = Infill volume - Tubelen*Area (- tip volume - almost no change) '''
TubeInnerDia = 2 * TubeDesignedWidth/np.pi # (mm)
TubeExpandedArea = (TubeDesignedWidth ** 2)/np.pi # (mm2)
print("Radius and cross-section area of the tube of the compact design: %.2fmm, %.2fmm2" % (TubeInnerDia, TubeExpandedArea))

#''' Infill Ratio (obsoleted) '''
# TotalArea = TubeLengthValues * TubeDesignedWidth + (TubeDesignedWidth + PouchBottomWidth) * PouchNeckDist / 2
# TriangleArea = PouchHeight * PouchBottomWidth / 2
# infillRatio = TriangleArea/TotalArea
# print("Tube Length (mm): ", TubeLengthValues)
# print("Infill Ratio: ", infillRatio)

# TubeExpandedArea = (TubeDesignedWidth ** 2)/np.pi # (mm2)
# TubeExpandedVolumes = TubeLengthValues * TubeExpandedArea / 1000 # (mL) converted volume 
# print("Tube Volume (mL): ", TubeExpandedVolumes)

In [ ]:
''' Condition table '''
# Layout: %d - number of dashes dividing rectangle column, T - extra triangle column, - R - an extra rectangle cell
colNames = ['CondiName','Shell','ls_um','Fluid','Infill_mL','DashLength_mm','FlatDashDist_mm','DashSpace_mm','Layout',
            'triNum','rectNum','TubeLength_mm','TubeInnerDia_mm','VoltPolar','Voltage_V','HVSupply','Info']

''' For data collected before 2022.04 '''
# condiTable = [
#     ['6mLBOPP12',           'BOPP',20,'IC',6,12,10,12.66,'6T5T4T3T2T1T',48,27,150,6-2,'Uni',8800,'Wearable','20210719_RigidTube'],
#     ['6mLBOPP16',           'BOPP',20,'IC',6,16,10,8.66,'6T5T4T3T2T1T',48,27,150,6-2,'Uni',8800,'Wearable','20210719_RigidTube'],
#     ['6mLL0Ws12SoftTube','PETL0Ws',30,'IC',6,12,10,12.66,'6T5T4T3T2T1T',48,27,150,4,'Uni',8800,'Wearable','20210719_SoftTube'],
#     ['BOPP16ShiftedU7kV',   'BOPP',20,'IC',6,16,10,8.66,'5T4T3T2T1TRT',36,21,150,6-2,'Uni',7200,'Wearable','20210729_RigidTube'],
#     ['L0Ws16ShiftedU9kV','PETL0Ws',30,'IC',6,16,10,8.66,'5T4T3T2T1TRT',36,21,150,6-2,'Uni',8800,'Wearable','20210729_RigidTube'],
#     ['L0Ws16NoShift',    'PETL0Ws',30,'IC',6,16,10,8.66,'T5T4T3T2T1TR',48,21,150,6-2,'Uni',8800,'Wearable','20210729_RigidTube'],
#     ['Mylar16',            'Mylar',15,'IC',6,16,10,8.66,'5T4T3T2T1TRT',36,21,150,6-2,'Uni',7200,'Wearable','20210728_RigidTube'],
#     ['Mylar16NoShift',     'Mylar',15,'IC',6,16,10,8.66,'T5T4T3T2T1TR',48,21,150,6-2,'Uni',7200,'Wearable','20210728_RigidTube'],
#     ['BOPP16',              'BOPP',20,'IC',6,16,10,8.66,'T5T4T3T2T1TR',48,21,150,6-2,'Uni',7200,'Wearable','20210728_RigidTube'],
#     ['BOPP20',              'BOPP',20,'IC',6,20,10,8.66,'T5T4T3T2T1',45,20,150,6-2,'Uni',7200,'Wearable','20210728_RigidTube'],
#     ['BOPP16U9kV',          'BOPP',20,'IC',6,16,10,8.66,'T5T4T3T2T1TR',48,21,150,6-2,'Uni',8800,'Wearable','20210728_RigidTube'],
#     ['MLSi16SB7kV',  'Mylar',15,'Silicone',6,16,10,8.66,'5T4T3T2T1TRT',36,21,150,6-2,'Bi',7000,'Trek','20210813_RigidTube_16S'],
#     ['MLSi16CB7kV',  'Mylar',15,'Silicone',6,16,10,8.66,'T5T4T3T2T1TR',48,21,150,6-2,'Bi',7000,'Trek','20210813_RigidTube_16C'],
#     ['MLSi20SB7kV',  'Mylar',15,'Silicone',6,20,10,8.66,'5T4T3T2T1T',37,20,150,6-2,'Bi',7000,'Trek','20210813_RigidTube_20S'],
#     ['MLSi20B7kV',   'Mylar',15,'Silicone',6,20,10,8.66,'T5T4T3T2T1',45,20,150,6-2,'Bi',7000,'Trek','20210813_RigidTube'],
#     ['Tube6mmPressBalloon','Mylar',15,'IC',6,20,10,8.66,'T5T4T3T2T1',45,20,35,6-2,'Bi',6900,'Wearable','20210924'],
#     ['MedTubePressBalloon','Mylar',15,'IC',6,20,10,8.66,'T5T4T3T2T1',45,20,150,3.2,'Bi',6900,'Wearable','20210924'],
#     ['PressBalloon2',      'Mylar',15,'IC',6,20,10,8.66,'T5T4T3T2T1',45,20,150,3.2,'Bi',6900,'Wearable','20210924'],
#     ['Tube8mmPressBalloon','Mylar',15,'IC',6,20,10,8.66,'T5T4T3T2T1',45,20,150,8-2,'Bi',6900,'Wearable','20210924'],
#     ['Pressure',           'Mylar',15,'IC',6,20,10,8.66,'T5T4T3T2T1',45,20,150,8-2,'Bi',6900,'Wearable','20210924'],
#     ['FR3Tube8mm',         'Mylar',15,'IC',6,20,10,8.66,'T5T4T3T2T1',45,20,150,8-2,'Bi',6900,'Wearable','20211008'],
#     ['FR3MedTube',         'Mylar',15,'IC',6,20,10,8.66,'T5T4T3T2T1',45,20,150,3.2,'Bi',6900,'Wearable','20211008'],
#     ['SiMedTube',    'Mylar',15,'Silicone',6,20,10,8.66,'T5T4T3T2T1',45,20,150,3.2,'Bi',6900,'Wearable','20211008'],
#     ['L0WsSi5-20Trek8kV',      'L0Ws',20,'Silicone',6,20,10,8.66,'T5T4T3T2T1',45,20,150,4.0,'Bi',8000,'Trek','20220302'],
#     ['L0WsSi5-20Trek9kV',      'L0Ws',20,'Silicone',6,20,10,8.66,'T5T4T3T2T1',45,20,150,4.0,'Bi',9000,'Trek','20220302'],
#     ['L0WsSi5-20Trek10kV',     'L0Ws',20,'Silicone',6,20,10,8.66,'T5T4T3T2T1',45,20,150,4.0,'Bi',10000,'Trek','20220302'],
#     ['L0WsSi6-16Trek9kVHVTop', 'L0Ws',20,'Silicone',6,16,10,8.66,'6T5T4T3T2T1T',48,27,150,4.0,'Bi',9000,'Trek','20220302'],
#     ['L0WsSi6-16Trek10kVHVTop','L0Ws',20,'Silicone',6,16,10,8.66,'6T5T4T3T2T1T',48,27,150,4.0,'Bi',10000,'Trek','20220302'],
#     ['L0WsSi5-20sTrek9kVHVBot','L0Ws',20,'Silicone',6,20,10,8.66,'5T4T3T2T1T',37,20,150,4.0,'Bi',9000,'Trek','20220302'],
# ]

''' For data collected since 2022.04 '''
# condiTable = [
#     ['MLSi20HV7kV.5Hz',     'Mylar',15,'Silicone',6,20,10,8.66,'T5T4T3T2T',45,18,150,4.0,'Bi',7000,'Trek','20220402'],
#     ['MLSi20HV7kV1Hz',      'Mylar',15,'Silicone',6,20,10,8.66,'T5T4T3T2T',45,18,150,4.0,'Bi',7000,'Trek','20220402'],
#     ['MLSi20HV7kV10mL1Hz',  'Mylar',15,'Silicone',10,20,10,8.66,'T5T4T3T2T',45,18,150,4.0,'Bi',7000,'Trek','20220402'],
#     ['MLSi20HV7kV10mL2Hz',  'Mylar',15,'Silicone',10,20,10,8.66,'T5T4T3T2T',45,18,150,4.0,'Bi',7000,'Trek','20220402'],
#     ['MLSi20HV7kV10mL4Hz',  'Mylar',15,'Silicone',10,20,10,8.66,'T5T4T3T2T',45,18,150,4.0,'Bi',7000,'Trek','20220402'],
#     ['MLSi20HV7kV10mL10Hz', 'Mylar',15,'Silicone',10,20,10,8.66,'T5T4T3T2T',45,18,150,4.0,'Bi',7000,'Trek','20220402'],
#     ['MLSi20HV7kV10mL20Hz', 'Mylar',15,'Silicone',10,20,10,8.66,'T5T4T3T2T',45,18,150,4.0,'Bi',7000,'Trek','20220402'],
    
#     ['MLSi20HV7kV6mL.5Hz',  'Mylar',15,'Silicone',6,20,10,8.66,'T5T4T3T2T',45,18,150,4.0,'Bi',7000,'Trek','20220402'],
#     ['MLSi20HV7kV7mL.5Hz',  'Mylar',15,'Silicone',7,20,10,8.66,'T5T4T3T2T',45,18,150,4.0,'Bi',7000,'Trek','20220402'],
#     ['MLSi20HV7kV8mL.5Hz',  'Mylar',15,'Silicone',8,20,10,8.66,'T5T4T3T2T',45,18,150,4.0,'Bi',7000,'Trek','20220402'],
#     ['MLSi20HV7kV10mL.5Hz', 'Mylar',15,'Silicone',10,20,10,8.66,'T5T4T3T2T',45,18,150,4.0,'Bi',7000,'Trek','20220402'],
    
#     ['MLSi16F4mLHV7kV',     'Mylar',15,'Silicone',4, 16,10,8.66,'6T5T4T3T2T1T',48,27,150,4.0,'Bi',7000,'Trek','20220426'],
#     ['MLSi16F5mLHV7kV',     'Mylar',15,'Silicone',5, 16,10,8.66,'6T5T4T3T2T1T',48,27,150,4.0,'Bi',7000,'Trek','20220426'],
#     ['MLSi16F6mLHV7kV',     'Mylar',15,'Silicone',6, 16,10,8.66,'6T5T4T3T2T1T',48,27,150,4.0,'Bi',7000,'Trek','20220426'],
#     ['MLSi16F7mLHV7kV',     'Mylar',15,'Silicone',7, 16,10,8.66,'6T5T4T3T2T1T',48,27,150,4.0,'Bi',7000,'Trek','20220426'],
#     ['MLSi16F8mLHV7kV',     'Mylar',15,'Silicone',8, 16,10,8.66,'6T5T4T3T2T1T',48,27,150,4.0,'Bi',7000,'Trek','20220426'],
#     ['MLSi16F9mLHV7kV',     'Mylar',15,'Silicone',9, 16,10,8.66,'6T5T4T3T2T1T',48,27,150,4.0,'Bi',7000,'Trek','20220426'],
#     ['MLSi16F10mLHV7kV',    'Mylar',15,'Silicone',10,16,10,8.66,'6T5T4T3T2T1T',48,27,150,4.0,'Bi',7000,'Trek','20220426'],
#     ['MLSi16F11mLHV7kV',    'Mylar',15,'Silicone',11,16,10,8.66,'6T5T4T3T2T1T',48,27,150,4.0,'Bi',7000,'Trek','20220426'],
#     ['MLSi16F12mLHV7kV',    'Mylar',15,'Silicone',12,16,10,8.66,'6T5T4T3T2T1T',48,27,150,4.0,'Bi',7000,'Trek','20220426'],

''' Cp: For data from compact actuator since 2022.06 '''
''' Bare:  bare actuator without any insulation, taped to table. 2022.08 '''
''' Ins:  insulated actuator with silicone gel. 2022.08 '''
''' 0.4 sec charge time '''
# condiTable = [
# #     ['MLSiCpPWM002', 'Mylar',15,'Silicone',1.8, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',2987,'Wearable','20220600'],
# #     ['MLSiCpPWM004', 'Mylar',15,'Silicone',1.8, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',3605,'Wearable','20220600'],
# #     ['MLSiCpPWM008', 'Mylar',15,'Silicone',1.8, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',4692,'Wearable','20220600'],
# #     ['MLSiCpPWM016', 'Mylar',15,'Silicone',1.8, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',6262,'Wearable','20220600'],
# #     ['MLSiCpPWM032', 'Mylar',15,'Silicone',1.8, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',7153,'Wearable','20220600'],
# #     ['MLSiCpPWM064', 'Mylar',15,'Silicone',1.8, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',7813,'Wearable','20220600'],
# #     ['MLSiCpPWM100', 'Mylar',15,'Silicone',1.8, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',7909,'Wearable','20220600'],
    
#     ['MLSiCpBare60mm', 'Mylar',15,'Silicone',1.1, 1,24.64,72.71,'Compact',1,0,60,4.55,'Uni',7000,'Trek','20220815'],
#     ['MLSiCpBare70mm', 'Mylar',15,'Silicone',1.15, 1,24.64,72.71,'Compact',1,0,70,4.55,'Uni',7000,'Trek','20220815'],
#     ['MLSiCpBare80mm', 'Mylar',15,'Silicone',1.2, 1,24.64,72.71,'Compact',1,0,80,4.55,'Uni',7000,'Trek','20220815'],
#     ['MLSiCpIns90mm', 'Mylar',15,'Silicone',1.8, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',7000,'Trek','20220815'],
#     ['MLSiCpBare100mm', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,100,4.55,'Uni',7000,'Trek','20220815'],
     
#     ['MLSiCpBr90mm',       'Mylar',15,'Silicone',1.3, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',7000,'Trek','20220830'],
#     ['MLSiCpBr110mm1.2mL', 'Mylar',15,'Silicone',1.2, 1,24.64,72.71,'Compact',1,0,110,4.55,'Uni',7000,'Trek','20220830'],
#     ['MLSiCpBr110mm1.4mL', 'Mylar',15,'Silicone',1.4, 1,24.64,72.71,'Compact',1,0,110,4.55,'Uni',7000,'Trek','20220830'],
#     ['MLSiCpBr110mm1.5mL', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,110,4.55,'Uni',7000,'Trek','20220830'],
#     ['MLSiCpBr110mm1.6mL', 'Mylar',15,'Silicone',1.6, 1,24.64,72.71,'Compact',1,0,110,4.55,'Uni',7000,'Trek','20220830'],
#     ['MLSiCpBr110mm1.8mL', 'Mylar',15,'Silicone',1.8, 1,24.64,72.71,'Compact',1,0,110,4.55,'Uni',7000,'Trek','20220830']
# ]

# ''' 1.2 sec charge time (infill should be 1.8mL instead of 1.5mL)'''
# condiTable = [
#     ['MLSiCpPWM002', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',1527,'Wearable','20220600'],
#     ['MLSiCpPWM004', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',2158,'Wearable','20220600'],
#     ['MLSiCpPWM008', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',3287,'Wearable','20220600'],
#     ['MLSiCpPWM016', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',4957,'Wearable','20220600'],
#     ['MLSiCpPWM032', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',5952,'Wearable','20220600'],
#     ['MLSiCpPWM064', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',6929,'Wearable','20220600'],
#     ['MLSiCpPWM100', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',7271,'Wearable','20220600']
# ]
# ''' 0.2 sec charge time (infill should be 1.8mL instead of 1.5mL)'''
# condiTable = [
#     ['MLSiCpPWM002', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',5123,'Wearable','20220600'],
#     ['MLSiCpPWM004', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',5755,'Wearable','20220600'],
#     ['MLSiCpPWM008', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',6645,'Wearable','20220600'],
#     ['MLSiCpPWM016', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',8139,'Wearable','20220600'],
#     ['MLSiCpPWM032', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',8819,'Wearable','20220600'],
#     ['MLSiCpPWM064', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',9135,'Wearable','20220600'],
#     ['MLSiCpPWM100', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,90,4.55,'Uni',8809,'Wearable','20220600']

'''
L: Tube Length (mm), from compact actuator data 2022.09 - 2022.10
V: Voltage level (kV), from compact actuator data 2022.09 - 2022.10
'''
condiTable = [
    ['MLSiL110V3', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,110,4.55,'Uni',3000,'Wearable','20220600'],
    ['MLSiL110V4', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,110,4.55,'Uni',4000,'Wearable','20220600'],
    ['MLSiL110V5', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,110,4.55,'Uni',5000,'Wearable','20220600'],
    ['MLSiL110V6', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,110,4.55,'Uni',6000,'Wearable','20220600'],
    ['MLSiL110V7', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,110,4.55,'Uni',7000,'Wearable','20220600'],
    
    ['MLSiL60V3', 'Mylar',15,'Silicone',1.0, 1,24.64,72.71,'Compact',1,0,60,4.55,'Uni',3000,'Wearable','20220600'],
    ['MLSiL60V4', 'Mylar',15,'Silicone',1.0, 1,24.64,72.71,'Compact',1,0,60,4.55,'Uni',4000,'Wearable','20220600'],
    ['MLSiL60V5', 'Mylar',15,'Silicone',1.0, 1,24.64,72.71,'Compact',1,0,60,4.55,'Uni',5000,'Wearable','20220600'],
    ['MLSiL60V6', 'Mylar',15,'Silicone',1.0, 1,24.64,72.71,'Compact',1,0,60,4.55,'Uni',6000,'Wearable','20220600'],
    ['MLSiL60V7', 'Mylar',15,'Silicone',1.0, 1,24.64,72.71,'Compact',1,0,60,4.55,'Uni',7000,'Wearable','20220600'],
    
    ['MLSiL100V3', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,100,4.55,'Uni',3000,'Wearable','20220600'],
    ['MLSiL100V4', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,100,4.55,'Uni',4000,'Wearable','20220600'],
    ['MLSiL100V5', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,100,4.55,'Uni',5000,'Wearable','20220600'],
    ['MLSiL100V6', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,100,4.55,'Uni',6000,'Wearable','20220600'],
    ['MLSiL100V7', 'Mylar',15,'Silicone',1.5, 1,24.64,72.71,'Compact',1,0,100,4.55,'Uni',7000,'Wearable','20220600'],
]

condiTable = pd.DataFrame(condiTable, columns = colNames)

# condiTable.loc[0:,'Infill_mL'] = condiTable.loc[0:,'Infill_mL'].multiply(infillRatio); print(condiTable['Infill_mL'])

condiTable.to_csv("ConditionTable.csv", index=False)

colNames2 = ['Material','DieConstant','DieStrength_V/um','Density_kg/m3','Viscosity_mm2/s']
materialProperty = [['BOPP',2.2,700,float('nan'),float('nan')],['Mylar',3.2,276,float('nan'),float('nan')],
                    ['L0Ws',3,float('nan'),float('nan'),float('nan')],
                    ['IC',3.2,20,920,34],['Silicone',2.6,15,920,5]]
materialProperty = pd.DataFrame(materialProperty, columns = colNames2)
# materialProperty.to_csv("MaterialProperty.csv", index=False)

In [ ]:
'''
General Functions
'''

def aPlot(figName='', is3D = False, dpi=72):
    ax = []
    
    fig1 = plt.figure(figsize = (6,3), dpi=dpi)
    
    fig1.suptitle(figName, fontsize=16)
    if(is3D):
        ax = fig1.add_subplot(111, projection='3d')
    else:
        ax = fig1.add_subplot(111)
        
    return ax, fig1

def sample2Time(ax, Fs): # Covert the xticks of current plot from samples to time using Fs
    locs, _ = plt.xticks()
    plt.xticks(locs, locs/Fs)
    ax.set_xlim(0, locs[-1])
    ax.set_xlabel('Time (secs)')

def lowpassFilter(datain, cutFreq, Fs, order):
    b, a = signal.butter(order, 2 * cutFreq / Fs, btype='low')
    dataout = signal.filtfilt(b, a, datain)
    return dataout

def decodeFileName(fileName):
    condi = re.split('_|\.csv', fileName)[3:5]
    names = condi[0]         
    trialNum = int(re.split('t', condi[1])[-1])
    
    return condi, names, trialNum

def decodeFileName2(fileName):
    condi = re.split('_|\.csv', fileName)[3:5]
    names = condi[0]         
    trialNum = int(re.split('t', condi[1])[-1])
    
    print(names)
    freq = None
    freqInfo = re.findall('[\.]*\d+Hz', names)
    if freqInfo:
        freq = float(re.split('Hz', freqInfo[0])[0])
        print("Signal frequency (Hz) = %.3f" % freq)
    
    return condi, names, trialNum, freq

''' Identify pressure increasing duration and compute time-averaged pressure '''
def processPressure(rawData, Fs, inactivePressure, cutOff=5, order=8):   
    print(rawData.shape)
    print(np.max(rawData))
    filtData = lowpassFilter(rawData, cutOff, Fs, order) - inactivePressure
        
    maxFiltPress = np.max(filtData) 
    print(maxFiltPress)
    
    samp = np.arange(len(rawData))
    ax0, _ = aPlot("Truncat")
    ax0.plot(samp, rawData -inactivePressure, color='tab:orange')
    ax0.plot(samp, filtData, color='tab:blue')
    
    zipStart = samp[0]
    zipEnd = samp[-1]
    
    try:
        zipStart = np.nonzero(filtData > 0.25 * maxFiltPress)[0][0]
        ax0.plot([zipStart,zipStart], [0, maxFiltPress], '--k')
    except:
        print("Unable to identify the start of zipping process")

    try:
        zipEnd = np.argmax(filtData)
        ax0.plot([zipEnd,zipEnd], [0, maxFiltPress], '--k')
    except:
        print("Unable to identify the end of zipping process")
        
    plt.show()
    
    avgPressure = np.mean(rawData[zipStart:zipEnd+1] - inactivePressure)
    
    deltaT = (zipEnd - zipStart) / Fs
    
    maxPressure = rawData[zipEnd] - inactivePressure # Maximum steady state pressure
    peakPressure = np.max(rawData) - inactivePressure # Peak pressure reachable
    
    return avgPressure, deltaT, peakPressure, np.array([zipStart, zipEnd])

''' 
(For different signal frequency of repeated measurement) 
Identify pressure increasing duration and compute time-averaged pressure 
'''
def cutRepeatTrial(rawData, Fs, inactivePressure, cutOff=5, order=8): 
    print(rawData.shape)
    maxRawValue = np.max(rawData)
    print("max value of raw data = %.3f" % maxRawValue)
    filtData = lowpassFilter(rawData, cutOff, Fs, order) - inactivePressure
    
    samp = np.arange(len(rawData))
    
    maxFiltPress = np.max(filtData) 
    print(maxFiltPress)
    
    segPointInd = np.squeeze(np.argwhere(filtData > 0.25 * maxFiltPress)) # Find value larger than 25% of peak as valid segment data point
    
    segGapInd = np.squeeze(np.argwhere(np.diff(segPointInd) > 1))# Index of point where large gap occurs (end and start of a seg)
    
    cutInd = (0.5 * (segPointInd[segGapInd] + segPointInd[segGapInd+1])).astype(int) # Cut in the middle of a end and a start point
    
    avgSegLen = np.mean(np.diff(cutInd))
    print("Avg. segment length = %d" % avgSegLen)
    
    cutInd = np.insert(cutInd, 0, max(cutInd[0]-avgSegLen, 0))
    
    ''' Plot data segmentation index '''
#     ax0, _ = aPlot("Cut Segments")
#     ax0.plot(samp, rawData -inactivePressure, color='tab:orange')
#     ax0.plot(samp[segPointInd], filtData[segPointInd], color='tab:blue')
#     ax0.plot(cutInd, np.zeros(cutInd.shape), '*g')
#     plt.show()
    return cutInd


In [ ]:
''' 
Design and zipping state (Rerun needed only when adding a new design layout) 
'''
colNames3 = ['Layout','triNum','rectNum','triNum1Expan','rectNum1Expan','triNum2Expan','rectNum2Expan']

stateLayout = [
    ['6T5T4T3T2T1T', 48,27, -12,-6, -23,-12],
    ['5T4T3T2T1TRT', 36,21, -12,-6, -21,-11],
    ['T5T4T3T2T1', 45,20, -10,-5, -20,-10],
    ['T5T4T3T2T1TR', 48,21, -12,-6, -23,-11],
    ['5T4T3T2T1T', 37,20, -10,-5, -19,-10],
    ['T5T4T3T2T', 45,18, -10,-4, -19,-8],
    ['Compact', 1,0, 0,0, 0,0]
]

stateLayout = pd.DataFrame(stateLayout, columns = colNames3)
# stateLayout.to_csv("StateLayout.csv", index=False)

''' 
Generate model by matched parameters after getting model data of Valid Arc Length 
(Rerun needed only when adding a new design layout) 
(Must add layout information to 'ValidArcLength' before execute 'EnergyModel')
(The basic 'ValidArcLength' table is produced by 'SearchValidArc.py')
'''
md = pd.read_csv("./data/ValidArcLength.csv")
dataLen = md.shape[0]
print("Data contains %d rows" % dataLen)

''' Get total volume of design structures for inactive and zipped state '''
stateLayout.head()

''' Compute total volume based on pouch design and zipping state '''
for layout_i in stateLayout['Layout']:
    ind = (stateLayout['Layout'] == layout_i)
    md[layout_i] = stateLayout.loc[ind,'triNum'].iloc[0] * md['triVol'] + stateLayout.loc[ind,'rectNum'].iloc[0] * md['rectVol']

md.to_csv("ValidArcLength.csv", index=False)
md.head(20)   

In [ ]:
'''
Code below is for segmenting data of fluid pressure measurement, not needed for the future.









Measurement [Syringe pump infill volume exp.] 2022.04.26 ten charge-discharge pulses cycles at 1 Hz. 
Sampling frequency of NIDAQ is caliberated at 1840 Hz
'''
measureDataPath = ".\Data220426"

pressData = []
Fs = 1840

sigFreq = 2
trialArray = (0.9+np.arange(1, 21))/sigFreq
beginInd = (Fs * np.array(0.1 + trialArray)).astype(int)
endInd = (Fs * np.array(0.1 + 1/sigFreq + trialArray)).astype(int)

for root, directories, files in walk(measureDataPath):
    for fileName in files:
         if (fileName[-3:] == 'csv'):    
                condi, names, trialNum = decodeFileName(fileName)
                
                data = np.genfromtxt(ospa.join(root, fileName), delimiter=',')
                
                data = (data[:,2] - calib[1]) * calib[0] # Unit conversion and only keep used channel (2), channel (0) is current
                
                dataMin = np.min(data) 
                if dataMin < 0:
                    data = data-dataMin # Rectify data

                inactivePressure = np.min(data[:int(0.5*Fs)])
                
#                 ax, _ = aPlot()
#                 ax.plot(data)
#                 ax.plot(beginInd, 0.02+np.zeros(beginInd.shape), '*r')
#                 ax.plot(endInd, 0.02+np.zeros(endInd.shape), '.g')
#                 plt.show()

                for i in range(len(beginInd)):             
                    avgPressure, deltaT, maxPressure, segInd = processPressure(data[beginInd[i]:endInd[i]], Fs, 
                                                                               inactivePressure, 20, 3)
                    
                    P0 = data[beginInd[i]+segInd[0]]-inactivePressure

                    print("\n%s - Trial%02d - Seg%d" % (names, trialNum, i+1))
                    print("Avg. Pressure of zipping = %.3f (bar), Max. P. = %.3f (bar) in %.3f (sec)\n" % 
                          (avgPressure, maxPressure, deltaT))
                    pressData.append([names, avgPressure*100, deltaT, maxPressure*100, P0*100,
                                    (data[beginInd[i]:beginInd[i]+segInd[1]+int(0.5*Fs)]-inactivePressure)*100, segInd, sigFreq]) # 1 bar = 100 kPa
                    
pressData = pd.DataFrame(pressData, columns = ['CondiName','AvgPressure_kPa','RiseTime_sec','MaxPressure_kPa',
                                               'P0_kPa','PressSig_kPa','SegIndex', 'SigFreq'])

sigData = pressData[['CondiName','PressSig_kPa','SegIndex']] 
     
pressData = pressData.drop(columns=['PressSig_kPa','SegIndex'])
pressData.to_csv("Measurements.csv")

In [ ]:
'''Measurement 2022 Duel Trial'''
# pressData = []
# Fs = 1000
# beginInd = ((np.array([1.5, 4]) - 0.1) * Fs).astype(int)
# endInd = ((np.array([3, 5.5]) + 0.1) * Fs).astype(int)

# for root, directories, files in walk(".\data\DualTrial"):
#     for fileName in files:
#          if (fileName[-3:] == 'csv'):    
#                 condi, names, trialNum = decodeFileName(fileName)
                
#                 data = np.genfromtxt(ospa.join(root, fileName), delimiter=',')
                
#                 data = (data[:,2] - calib[1]) * calib[0] # Unit conversion and only keep used channel (2), channel (0) is current
                
#                 dataMin = np.min(data) 
#                 if dataMin < 0:
#                     data = data-dataMin # Rectify data

#                 inactivePressure = np.min(data[:int(0.5*Fs)])

#                 for i in range(len(beginInd)): 
#                     avgPressure, deltaT, maxPressure, segInd = processPressure(data[beginInd[i]:endInd[i]], Fs, inactivePressure)
#                     P0 = data[beginInd[i]+segInd[0]]-inactivePressure

#                     print("\n%s - Trial%02d - Seg%d" % (names, trialNum, i+1))
#                     print("Avg. Pressure of zipping = %.3f (bar), Max. P. = %.3f (bar) in %.3f (sec)\n" % 
#                           (avgPressure, maxPressure, deltaT))
#                     pressData.append([names, avgPressure*100, deltaT, maxPressure*100, P0*100,
#                                     (data[beginInd[i]:beginInd[i]+segInd[1]+int(0.5*Fs)]-inactivePressure)*100, segInd]) # 1 bar = 100 kPa


In [ ]:
'''Measurement 2022 Twenty Trial (Act4 Act6, with frequency of 0.5Hz, 1Hz)'''
'''
Shifted0.5s: 1-3, 5-7, 9-11, ...
[1, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, 53, 57, 61, 65, 69, 73, 77]
'''

pressData = []
Fs = 2000

# measureDataPath = ".\Data220401Relabelled\ValidData"
measureDataPath = ".\Data220401Relabelled\Act6Volume"

for root, directories, files in walk(measureDataPath):
    for fileName in files:
         if (fileName[-3:] == 'csv'):    
                condi, names, trialNum, sigFreq = decodeFileName2(fileName)
                
                trialArray = np.arange(1, 40, 2.22)/sigFreq
                beginInd = (Fs * np.array(0.1 + trialArray)).astype(int)
                endInd = (Fs * np.array(0.1 + 2/sigFreq + trialArray)).astype(int)
                
                data = np.genfromtxt(ospa.join(root, fileName), delimiter=',')
                
                data = (data[:,2] - calib[1]) * calib[0] # Unit conversion and only keep used channel (2), channel (0) is current
                
                dataMin = np.min(data) 
                if dataMin < 0:
                    data = data-dataMin # Rectify data

                inactivePressure = np.min(data[:int(0.5*Fs)])
                
#                 ax, _ = aPlot()
#                 ax.plot(data)
#                 ax.plot(beginInd, np.zeros(beginInd.shape), '*r')
#                 ax.plot(endInd, np.zeros(endInd.shape), '*g')
#                 plt.show()

                for i in range(len(beginInd)): 
#                     ax.plot(beginInd[i]+np.arange(len(data[beginInd[i]:endInd[i]])), data[beginInd[i]:endInd[i]])
                    avgPressure, deltaT, maxPressure, segInd = processPressure(data[beginInd[i]:endInd[i]], Fs, 
                                                                               inactivePressure, 20, 8)
                    P0 = data[beginInd[i]+segInd[0]]-inactivePressure

                    print("\n%s - Trial%02d - Seg%d" % (names, trialNum, i+1))
                    print("Avg. Pressure of zipping = %.3f (bar), Max. P. = %.3f (bar) in %.3f (sec)\n" % 
                          (avgPressure, maxPressure, deltaT))
                    pressData.append([names, avgPressure*100, deltaT, maxPressure*100, P0*100,
                                    (data[beginInd[i]:beginInd[i]+segInd[1]+int(0.5*Fs)]-inactivePressure)*100, segInd, sigFreq]) # 1 bar = 100 kPa
                    
pressData = pd.DataFrame(pressData, columns = ['CondiName','AvgPressure_kPa','RiseTime_sec','MaxPressure_kPa',
                                               'P0_kPa','PressSig_kPa','SegIndex', 'SigFreq'])

sigData = pressData[['CondiName','PressSig_kPa','SegIndex']] 
     
pressData = pressData.drop(columns=['PressSig_kPa','SegIndex'])
pressData.to_csv("Measurements.csv")

In [ ]:
'''Measurement 2022 Frequency comparison'''
pressData = []
Fs = 2000

for root, directories, files in walk(".\Data220401Relabelled\Act6Freq"):
    for fileName in files:
         if (fileName[-3:] == 'csv'):    
                condi, names, trialNum, sigFreq = decodeFileName2(fileName)
                
                trialArray = np.arange(1, 40, 2.22)/sigFreq
                beginInd = (Fs * np.array(0.1 + trialArray)).astype(int)
                endInd = (Fs * np.array(0.1 + 2/sigFreq + trialArray)).astype(int)
                
                data = np.genfromtxt(ospa.join(root, fileName), delimiter=',')
                
                data = (data[:,2] - calib[1]) * calib[0] # Unit conversion and only keep used channel (2), channel (0) is current
                
                dataMin = np.min(data) 
                if dataMin < 0:
                    data = data-dataMin # Rectify data

                inactivePressure = np.min(data[:int(0.5*Fs)])
                
                cutInd = cutRepeatTrial(data, Fs, inactivePressure, 20, 8)           
                
                for i in range(len(cutInd)-1): 
                    avgPressure, deltaT, maxPressure, segInd = processPressure(data[cutInd[i]:cutInd[i+1]], Fs, 
                                                                               inactivePressure, 100, 10)
                    P0 = data[cutInd[i]+segInd[0]]-inactivePressure

                    print("\n%s - Trial%02d - Seg%d" % (names, trialNum, i+1))
                    print("Avg. Pressure of zipping = %.3f (bar), Max. P. = %.3f (bar) in %.3f (sec)\n" % 
                          (avgPressure, maxPressure, deltaT))
                    pressData.append([names, avgPressure*100, deltaT, maxPressure*100, P0*100,
                                    (data[cutInd[i]:cutInd[i]+int(1.5*segInd[1])] - inactivePressure)*100, segInd, sigFreq]) # 1 bar = 100 kPa
                    
                ''' Plot the Entire Pressure Signal '''
                if False:
                    ax,_ = aPlot(dpi = 300)
                    ax.plot(data - inactivePressure,color='tab:red')
                    ax.set_ylabel('Pressure (kPa)')
                    sample2Time(ax, Fs)
                    plt.show()
                    

        
# pressData = pd.DataFrame(pressData, columns = ['CondiName','AvgPressure_kPa','RiseTime_sec','MaxPressure_kPa',
#                                                'P0_kPa','PressSig_kPa','SegIndex', 'SigFreq'])

# sigData = pressData[['CondiName','PressSig_kPa','SegIndex']] 
     
# pressData = pressData.drop(columns=['PressSig_kPa','SegIndex'])
# pressData.to_csv("Measurements.csv")


# ''' Incorrect Index? '''
# for i in sigData.index:
#     ax,_=aPlot()
#     ax.plot(sigData.loc[i,'PressSig_kPa'],color='tab:red')
#     ax.plot([sigData.loc[i,'SegIndex'][0],sigData.loc[i,'SegIndex'][0]], [0, 10], '--k')
#     ax.plot([sigData.loc[i,'SegIndex'][1],sigData.loc[i,'SegIndex'][1]], [0, 10], '--k')
#     ax.set_ylabel('Pressure_kPa')

In [ ]:
'''Measurement 2021'''
# Fs = 1000

# pressData = []
# ''' Process measurement data saved as individual csv files by trial '''
# for root, directories, files in walk(".\data\ValidData"):
#     for fileName in files:
#          if (fileName[-3:] == 'csv'):       
#             condi, names, trialNum = decodeFileName(fileName)

#             data = np.genfromtxt(ospa.join(root, fileName), delimiter=',')
            
#             data = (data - calib[1]) * calib[0] # Unit conversion       
#             dataMin = np.min(data) 
#             if dataMin < 0:
#                 data = data-dataMin # Rectify data
            
#             inactivePressure = np.min(data[:int(0.5*Fs)])

#             avgPressure, deltaT, maxPressure, segInd = processPressure(data, Fs, inactivePressure)
#             P0 = data[segInd[0]]-inactivePressure
            
#             print("\n%s - Trial%02d" % (names, trialNum))
#             print("Avg. Pressure of zipping = %.3f (bar), Max. P. = %.3f (bar) in %.3f (sec)\n" % 
#                   (avgPressure, maxPressure, deltaT)) 
#             pressData.append([names, avgPressure*100, deltaT, maxPressure*100, P0*100,
#                              (data[:segInd[1]+int(0.5*Fs)]-inactivePressure)*100, segInd]) # 1 bar = 100 kPa

# ''' Process data switch three times with reversed polarity from bipolar HV Trek box '''
# beginInd = ((np.array([1, 8, 15]) - 0.1) * Fs).astype(int)
# endInd = ((np.array([5, 12, 19]) + 0.1) * Fs).astype(int)
# for root, directories, files in walk(".\data\TrekData"):
#     for fileName in files:
#          if (fileName[-3:] == 'csv'):
#             condi, names, trialNum = decodeFileName(fileName)

#             data = np.genfromtxt(ospa.join(root, fileName), delimiter=',')
            
#             data = (data - calib[1]) * calib[0] # Unit conversion       
#             dataMin = np.min(data) 
#             if dataMin < 0:
#                 data = data-dataMin # Rectify data

#             inactivePressure = np.min(data[:int(0.5*Fs)])

#             for i in range(len(beginInd)): 
#                 avgPressure, deltaT, maxPressure, segInd = processPressure(data[beginInd[i]:endInd[i]], Fs, inactivePressure)
#                 P0 = data[beginInd[i]+segInd[0]]-inactivePressure
                
#                 print("\n%s - Trial%02d - Seg%d" % (names, trialNum, i+1))
#                 print("Avg. Pressure of zipping = %.3f (bar), Max. P. = %.3f (bar) in %.3f (sec)\n" % 
#                       (avgPressure, maxPressure, deltaT))
#                 pressData.append([names, avgPressure*100, deltaT, maxPressure*100, P0*100,
#                                 (data[beginInd[i]:beginInd[i]+segInd[1]+int(0.5*Fs)]-inactivePressure)*100, segInd]) # 1 bar = 100 kPa

# ''' Process data driven by reversed polarity from bipolar wearable controller '''                
# for root, directories, files in walk(".\data\ValidData2"):
#     for fileName in files:
#          if (fileName[-3:] == 'csv'):       
#             condi, names, trialNum = decodeFileName(fileName)
#             names = re.split('U6900V', names)[0] # Shorten the names

#             data = np.genfromtxt(ospa.join(root, fileName), delimiter=',')     
            
#             data = (data[:,1] - calib[1]) * calib[0] # Unit conversion and only keep used channel (1)
            
#             dataMin = np.min(data) 
#             if dataMin < 0:
#                 data = data-dataMin # Rectify data
            
#             inactivePressure = np.min(data[:int(0.5*Fs)])

#             avgPressure, deltaT, maxPressure, segInd = processPressure(data[:int(3.5*Fs)], Fs, inactivePressure)
#             P0 = data[segInd[0]]-inactivePressure
            
#             print("\n%s - Trial%02d" % (names, trialNum))
#             print("Avg. Pressure of zipping = %.3f (bar), Max. P. = %.3f (bar) in %.3f (sec)\n" % 
#                   (avgPressure, maxPressure, deltaT)) 
#             pressData.append([names, avgPressure*100, deltaT, maxPressure*100, P0*100,
#                              (data[:segInd[1]+int(0.5*Fs)]-inactivePressure)*100, segInd]) # 1 bar = 100 kPa

# ''' Process data switch two times with reversed polarity from bipolar HV Trek box (2022) '''
# beginInd = ((np.array([1.5, 4]) - 0.1) * Fs).astype(int)
# endInd = ((np.array([3, 5.5]) + 0.1) * Fs).astype(int)
# for root, directories, files in walk(".\data\TrekData2"):
#     for fileName in files:
#          if (fileName[-3:] == 'csv'):
#             condi, names, trialNum = decodeFileName(fileName)

#             data = np.genfromtxt(ospa.join(root, fileName), delimiter=',')
            
#             data = (data[:,2] - calib[1]) * calib[0] # Unit conversion and only keep used channel (2) 
#             dataMin = np.min(data) 
#             if dataMin < 0:
#                 data = data-dataMin # Rectify data

#             inactivePressure = np.min(data[:int(0.5*Fs)])

#             for i in range(len(beginInd)): 
#                 avgPressure, deltaT, maxPressure, segInd = processPressure(data[beginInd[i]:endInd[i]], Fs, inactivePressure)
#                 P0 = data[beginInd[i]+segInd[0]]-inactivePressure # To remove DC pressure
                
#                 print("\n%s - Trial%02d - Seg%d" % (names, trialNum, i+1))
#                 print("Avg. Pressure of zipping = %.3f (bar), Max. P. = %.3f (bar) in %.3f (sec)\n" % 
#                       (avgPressure, maxPressure, deltaT))
#                 pressData.append([names, avgPressure*100, deltaT, maxPressure*100, P0*100,
#                                 (data[beginInd[i]:beginInd[i]+segInd[1]+int(0.5*Fs)]-inactivePressure)*100, segInd]) # 1 bar = 100 kPa
                                      
# pressData = pd.DataFrame(pressData, columns = ['CondiName','AvgPressure_kPa','RiseTime_sec','MaxPressure_kPa',
#                                                'P0_kPa','PressSig_kPa','SegIndex'])

# for aCondi in condiTable['CondiName']: # Drop any extra measurement more than five trials
#     allTrials = pressData[pressData['CondiName'] == aCondi]
#     trialNum = allTrials.shape[0]
#     if trialNum > 5:
#         pressData = pressData.drop(pressData[pressData['CondiName'] == aCondi].index)
#         pressData = pressData.append(allTrials.iloc[:5])

# sigData = pressData[['CondiName','PressSig_kPa','SegIndex']] 
     
# pressData = pressData.drop(columns=['PressSig_kPa','SegIndex'])
# pressData.to_csv("Measurements.csv")



In [ ]:
print(sigData.head())
simData = pd.read_csv("./data./temp_result.csv") # Data from model simulation
simData.tail()

In [ ]:
''' New estimation of air volume (2022.04.06) '''
airV0 = 1e-6 # Unit: m3, Initial air volume is roughly estimated to be 1? mL. (volume of inner tube is around 1.8 mL)
 
''' By law of P0V0 = P1V1, with P0 and P1 took from the measurement'''
avgPressLoss = []
for i in sigData.index:
#     airV0 = simData.loc[simData['CondiName'] == sigData.loc[i,'CondiName'], 'dV1Expan_m3'].iloc[0] # Incorrect Assume all oil pushed to tube
    
    i0 = sigData.loc[i,'SegIndex'][0] # Starting index at lowest pressure
    try:
        i1 = np.argwhere(sigData.loc[i,'PressSig_kPa'] == pressData.loc[i,'MaxPressure_kPa'])[0][0] # End index at max pressure
    except:
        i1 = sigData.loc[i,'SegIndex'][1]
    P0 = pressData.loc[i,'P0_kPa']
    P1 = pressData.loc[i,'MaxPressure_kPa']
    
    airV1 = airV0 * P0/P1
    print("%s - For pressure increased from P0 = %.3f kPa to P1 = %.3f kPa" % (sigData.loc[i,'CondiName'],
          pressData.loc[i,'P0_kPa'], pressData.loc[i,'MaxPressure_kPa']))
    print("Time-average pressure = %.3f kPa (gap to max. = %.3f kPa)" % (pressData.loc[i,'AvgPressure_kPa'],
            pressData.loc[i,'MaxPressure_kPa'] - pressData.loc[i,'AvgPressure_kPa']))
    print("Compressed-air steady state volume = %.6f mL" % (airV1 * 1e6))
    
    filtPressure = lowpassFilter(sigData.loc[i,'PressSig_kPa'], 30, Fs, 6) # Smooth volume change before division
    airVi = airV1 * P1 / filtPressure[i0 : i1]  # Unit: m3
    
    Qi = np.diff(airVi) * Fs # Flow rate, Unit: m3/s
    
    PiLoss = simData.loc[simData['CondiName'] == sigData.loc[i,'CondiName'], 'lossFactor'].iloc[0] * Qi *1e-3 # Unit: kPa

    avgPLoss = np.mean(PiLoss) # Unit: kPa
    print("time avg. flow rate = %.3f mL/s and pressure loss = %.3f kPa" % (np.mean(Qi) * 1e6, avgPLoss))
    
    avgPressLoss.append(avgPLoss)
    
    if pressData.loc[i,'MaxPressure_kPa'] > 4: # Check signal 
        ax,_ = aPlot(dpi = 72)
        ax.plot(sigData.loc[i,'PressSig_kPa'],color='tab:red')
        
#         ax.plot(np.arange(i0+1,i1), PiLoss,color='tab:orange')
        
        ax.plot([i0, i0], [0, pressData.loc[i,'P0_kPa']], '-k')
        ax.plot([i1, i1], [0, pressData.loc[i,'MaxPressure_kPa']], '-k')
#         ax.plot([sigData.loc[i,'SegIndex'][1],sigData.loc[i,'SegIndex'][1]], [0, pressData.loc[i,'MaxPressure_kPa']], '--k')
        ax.set_ylabel('Pressure (kPa)')
        
#         ax2 = ax.twinx() 
#         ax2.plot(np.arange(i0,i1), airVi,color='tab:blue')

        sample2Time(ax, Fs)
        
        plt.show()
    
pressData['P1ExpanLoss_kPa'] = avgPressLoss
pressData.to_csv("Measurements.csv")     
    

In [ ]:
'''
Such estimation of pressure loss in tube is very inaccurate due to incorrect estimation of air volume
'''
''' Both Simulation and Measurement Data required '''
# print("Sampling frequency of current dataset = %.3f Hz" % Fs)

# sigData['V0'] = 0

# avgPressLoss = []
# for i in sigData.index:
#     deltaVolume = simData.loc[simData['CondiName'] == sigData.loc[i,'CondiName'], 'dV1Expan_m3'].iloc[0]

#     sigData.loc[i,'V0'] = deltaVolume / (1 - pressData.loc[i,'P0_kPa']/pressData.loc[i,'MaxPressure_kPa']) # Unit: m3
    
#     print("Initial Volume (mL) = %f" % (sigData.loc[i,'V0']*1e6))
#     print("Initial Pressure (kPa) = %f" % pressData.loc[i,'P0_kPa'])
#     print(sigData.loc[i,'SegIndex'])
    
#     print("%s inital air volume = %.6f mL" % (sigData.loc[i,'CondiName'], sigData.loc[i,'V0']*1e6)) 
    
#     ''' Realtime volume '''
# #     filteredPressure = lowpassFilter(sigData.loc[i,'PressSig_kPa'], 20, Fs, 8) # Smooth volume change before differentiation
    
    
#     Vi = sigData.loc[i,'V0'] * pressData.loc[i,'P0_kPa'] / (sigData.loc[i,'PressSig_kPa'] + 1e-3) # Unit: m3
    
#     zippingVi = Vi[sigData.loc[i,'SegIndex'][0]:sigData.loc[i,'SegIndex'][1]] # Unit: m3
    
# #     zippingVi = lowpassFilter(zippingVi, 20, Fs, 8) # Smooth volume change before differentiation
    
#     ''' Realtime volume flow rate '''
#     Qi = np.diff(zippingVi) * Fs # Unit: m3/s
#     avgQ = np.mean(Qi)
#     avgQRough = -deltaVolume * Fs/(sigData.loc[i,'SegIndex'][1] - sigData.loc[i,'SegIndex'][0])
    
#     print("Average flow rate estimated from realtime pressure = %.3f mL/s and from deltaV/zipTime = %.3f mL/s" %
#          (avgQ * 1e6, avgQRough * 1e6))
    
#     '''
#     Two methods: compute from realtime flow rate vs. volume change divided by rising time
#     '''
#     PiLoss = simData.loc[simData['CondiName'] == sigData.loc[i,'CondiName'], 'lossFactor'].iloc[0] * Qi *1e-3 # Unit: kPa 
    
#     avgPLoss = simData.loc[simData['CondiName'] == sigData.loc[i,'CondiName'], 'lossFactor'].iloc[0] * avgQ *1e-3 # Unit: kPa  
#     if (avgPLoss > 0):
#         print("Incorrect Pressure loss")
#     else:
#         print("%s avg. pressure loss = %.2f kPa" % (sigData.loc[i,'CondiName'], avgPLoss))
        
#     avgPLossRough = simData.loc[simData['CondiName'] == sigData.loc[i,'CondiName'], 'lossFactor'].iloc[0]*avgQRough*1e-3 # Unit: kPa    
#     if (avgPLossRough > 0):
#         print("Incorrect Pressure loss (Rough)")
#     else:
#         print("%s avg. pressure loss = %.2f kPa (Rough)" % (sigData.loc[i,'CondiName'], avgPLossRough))    
        
        
#     avgPressLoss.append(avgPLoss)
    
#     segX = np.arange(sigData.loc[i,'SegIndex'][0], sigData.loc[i,'SegIndex'][1]) # For zipping process
#     ax,_=aPlot()
# #     ax.plot(segX, zippingVi,'tab:blue')
#     ax.plot(segX[1:], Qi * 1e6,'tab:blue')
#     ax.plot([sigData.loc[i,'SegIndex'][0],sigData.loc[i,'SegIndex'][0]], [0, 6e-6], '--k')
#     ax.plot([sigData.loc[i,'SegIndex'][1],sigData.loc[i,'SegIndex'][1]], [0, 6e-6], '--k')
# #     ax.set_ylabel('blue: Volume (m3)')
#     ax.set_ylabel('blue: Flow rate (mL/s)')
#     ax2 = ax.twinx() 
#     ax2.plot(segX[1:], PiLoss,color='tab:orange')
#     ax2.plot(sigData.loc[i,'PressSig_kPa'],color='tab:red')
#     ax2.set_ylabel('orange: loss and\nred: measured\nPressure (kPa)')
#     plt.show();
    
# pressData['P1ExpanLoss_kPa'] = avgPressLoss
# pressData.to_csv("Measurements.csv") 